# Installs

In [ ]:
!git clone https://github.com/SungFeng-Huang/SSL-pretraining-separation.git

In [ ]:
pip install asteroid

In [4]:
pip install musdb

# Imports

In [ ]:
import asteroid
import sys
import os
sys.path.append(os.path.abspath('./SSL-pretraining-separation'))
from src.models import sepformer_tasnet
from asteroid.data import MUSDB18Dataset
import torch
import musdb

# Dataset

In [6]:
#Downloading dataset
db = musdb.DB(download=True)

Done!


In [ ]:
#Converting dataset from stem.mp4 files to wav files to make compatible with asteroid
!musdbconvert "../root/MUSDB18/MUSDB18-7" "../root/MUSDB18/MUSDB18-7-wav"

In [14]:
#torch dataset objects
train_dataset = MUSDB18Dataset(root='../root/MUSDB18/MUSDB18-7-wav', suffix='.wav', split='train')
test_dataset = MUSDB18Dataset(root='../root/MUSDB18/MUSDB18-7-wav', suffix='.wav', split='test')

94it [00:00, 2069.42it/s]
50it [00:00, 2037.47it/s]


In [16]:
#dataloaders which load batches of our datasets during training and validation
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1)
test = torch.utils.data.DataLoader(test_dataset, batch_size=1)

# Model

In [17]:
#Initialize model
model = sepformer_tasnet.SepFormerTasNet(n_src=4)

# Encoder

# Masking Network

# Decoder